In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [2]:
print("have compute_component_scores?", 'compute_component_scores' in globals())

have compute_component_scores? True


In [3]:
# ==== BUMBLE CONFIG ====

# Candidate pool / retrieval
BUMBLE_POOL_K      = 5000    # how many candidates to fetch from get_matches for the user
BUMBLE_MAX_KM      = 50      # geo radius for get_matches
BUMBLE_TOP_K       = 20      # how many recs to return

# Scoring blend (used later with compute_component_scores output)
# We'll combine: text_sim, age_score, lifestyle_score, loc_score
BUMBLE_WEIGHTS = {
    "text": 0.70,   # semantic similarity (or 'score' mapped to text_sim)
    "age":  0.10,   # age compatibility
    "life": 0.15,   # lifestyle alignment (drinks/smokes/drugs/diet if available)
    "loc":  0.05,   # distance score
}

# Messaging behavior
BUMBLE_OPENER_N          = 3       # how many suggested openers to generate per match

# Reproducibility for any sampling we do in this notebook
BUMBLE_RANDOM_SEED = 123

In [6]:
def ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [ ]:
def bumble_rank_basic(user_id: int,k: int = BUMBLE_TOP_K,pool_k: int = BUMBLE_POOL_K,
                      max_km: float = BUMBLE_MAX_KM,weights: dict | None = None) -> pd.DataFrame:
    """
    1) Pull candidates with hard gates via get_matches.
    2) compute_component_scores to get text/age/lifestyle/loc scores.
    3) Blend into bumble_score using your apply_weights if available (or manual fallback).
    4) Return top-k ranked table.
    """
    # 1) candidate pool (hard filters only)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True, use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True).copy()

    # 2) component scores
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
        drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
    cands = ensure_text_sim(cands)

    # 3) blend to final score
    ranked = apply_weights(cands, w_text=weights["text"], w_age=weights["age"],w_life=weights["life"], 
                           w_loc=weights["loc"]).copy()

    # ensure expected columns exist
    need_cols = ["user_id","final_score","text_sim","age_score","lifestyle_score","loc_score",
                 "distance_km","location","age","sex","orientation"]
    rename final-score to bumble score
    for c in need_cols:
        if c not in ranked.columns:
            ranked[c] = np.nan

    return ranked[need_cols].sort_values("bumble_score", ascending=False).head(k).reset_index(drop=True)